https://www.sfu.ca/~ssurjano/holder.html
https://www.sfu.ca/~ssurjano/crossit.html

In [694]:
import numpy as np
import random
import json

from typing import List, Tuple, Callable

import binary.binary_utils as bu
import mutation as mu

In [695]:
with open('config.json', 'r') as f:
    config = json.load(f)

In [696]:
def cross_in_tray(x1: int, x2: int) -> int:
    term1 = np.sin(x1) * np.sin(x2)
    term2 = np.exp(abs(100 - np.sqrt(x1 ** 2 + x2 ** 2) / np.pi))

    return -0.0001 * (abs(term1 * term2) + 1) ** 0.1

In [697]:
def holder_table(x1: int, x2: int) -> int:
    term1 = np.sin(x1) * np.cos(x2)
    term2 = np.exp(abs(1 - (np.sqrt(x1 ** 2 + x2 ** 2)) / np.pi))

    return -1 * abs(term1 * term2)

In [698]:
optimization_f = cross_in_tray

In [699]:
def fitness(individual: str, optimization_f: Callable = None) -> float:
    if optimization_f is None:
        optimization_f = holder_table
    
    if isinstance(individual, str):
        x1, x2 = bu.decode(individual, config["bits_per_input"]) 
    else:
        x1, x2 = individual
    
    return optimization_f(x1, x2)

In [700]:
def generate_initial_population(population_size: int, generate_f: Callable) -> List[dict]:
    generation = []
    
    for _ in range(population_size):
        individual = generate_f()
        
        fitness_score = fitness(individual, optimization_f)
            
        generation.append({"binary": individual, "score": fitness_score})
    return generation

In [701]:
def tournament_selection(population, tournament_size=3):
    tournament = random.sample(population, tournament_size)

    return min(tournament, key=lambda ind: ind["score"])

In [702]:
population = generate_initial_population(
    config["population_size"], 
    generate_f=lambda: bu.generate_binary_individual(config["bits_per_input"])
)
best_overall = min(population, key=lambda ind: ind["score"])
crossover_f = mu.one_point_crossover
for gen in range(config["generations"]):
    new_population = []
    elite = min(population, key=lambda ind: ind["score"])
    new_population.append(elite)
    
    while len(new_population) < config["population_size"]:
        parent1 = tournament_selection(population)
        parent2 = tournament_selection(population)
        
        if random.random() < config["crossover_rate"]:
            child1_binary, child2_binary = crossover_f(parent1["binary"], parent2["binary"])
        else: 
            child1_binary, child2_binary = parent1["binary"], parent2["binary"]
            
        child1_binary = bu.binary_mutation(child1_binary, config["mutation_rate"])
        child2_binary = bu.binary_mutation(child2_binary, config["mutation_rate"])
        
        child1 = {"binary": child1_binary, "score": fitness(child1_binary, optimization_f)}
        new_population.append(child1)
        
        if len(new_population) < config["population_size"]:
            child2 = {"binary": child2_binary, "score": fitness(child2_binary, optimization_f)}
            new_population.append(child2)
            
    population = new_population
    
    current_best = min(population, key=lambda ind: ind["score"])
    if current_best["score"] < best_overall["score"]:
        best_overall = current_best
    
    if (gen + 1) % 10 == 0:
        best_x1, best_x2 = bu.decode(best_overall["binary"], config["bits_per_input"])
        print(f"Generation {gen + 1}, Best score: {best_overall['score']:.6f}, x1={best_x1:.4f}, x2={best_x2:.4f}")
         
best_x1, best_x2 = bu.decode(best_overall["binary"], config["bits_per_input"])
print(f"Score: {best_overall['score']:.10f}")
print(f"x1={best_x1:.6f}, x2={best_x2:.6f}")
print(f"Binary: {best_overall['binary']}")

Generation 10, Best score: -2.052113, x1=-1.6078, x2=1.5294
Generation 20, Best score: -2.062496, x1=-1.3725, x2=1.3725
Score: -2.0624962014
x1=-1.372549, x2=1.372549
Binary: 1001000101101110
